# Cut Down Vocabulary for Full ArXiv Corpus

Use the full corpus, but cut down to a more reasonable number of tokens (say, 20,000).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import copy
import gensim
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import sys

sys.path.append('../../')
import tmx

## Dictionary and Corpus

In [ ]:
C_ROOT = '../../corpora/arxiv/arxiv-full-20M-spacy/'

### Dictionary

In [ ]:
dct = tmx.utils.unpickle(os.path.join(C_ROOT, 'arxiv-full-20M-spacy.mm.dictionary.cpickle'))

Most-used tokens:

In [ ]:
def make_token_df(dictionary):
    df = pd.DataFrame([{
        'token': dictionary[t],
        'docs': dictionary.dfs[t],
        'docs_frac': dictionary.dfs[t]/dictionary.num_docs
    } for t in dictionary])
    return df.sort_values('docs', ascending=False)

In [ ]:
tokens = make_token_df(dct)

In [ ]:
tokens.head(n=20)

### Corpus

In [ ]:
arxiv = gensim.corpora.MmCorpus(os.path.join(C_ROOT, 'arxiv-full-20M-spacy.mm'))

## Dictionary Thinning: Huge Corpus

In [ ]:
new_dct = copy.deepcopy(dct)

### Top 100K

Start by only keeping the top 100,000 tokens. We'll whittle it down from there.

In [ ]:
new_dct.filter_extremes(no_above=1.0, keep_n=100000)

In [ ]:
# New token list
tokens = make_token_df(new_dct)

In [ ]:
tokens['docs_frac'].plot.hist(bins=50, log='y')

In [ ]:
tokens.head(20)

### Eliminate Additional Stop Words

Everything over 50% that seems mostly contentless.

In [ ]:
# Remove most-common tokens that seem contentless
addl_stopwords = set(tokens[tokens['docs_frac'] >= 0.5]['token'])

In [ ]:
# Looking through the top tokens and keeping interesting ones
addl_stopwords.difference_update({
    'analysis', 'constant', 'datum', 'energy', 'field', 'finite', 'independent', 'limit', 
    'linear', 'mean', 'measure', 'model', 'parameter', 'point', 'range', 'scale', 'second', 
    'set', 'space', 'state', 'structure', 'time'})

In [ ]:
new_dct.filter_tokens(bad_ids=[i for i in new_dct if new_dct[i] in addl_stopwords])

### Eliminate Words with Little Content

 - Most words under three characters
 - Numbers
 - Words with a lot of punctuation

Short words. We keep interesting ones from the top 200, but we miss variants where punctuation is attached.

In [ ]:
sw_keep = {'1d', '2d', '3d', 'act', 'add', 'age', 'agn', 'ago', 'aid', 'aim', 'air', 'arc', 'arm', 
           'art', 'ask', 'au', 'bad', 'bar', 'big', 'bin', 'bit', 'box', 'cm', 'cut', 'day', 'dc', 
           'die', 'doe', 'dot', 'end', 'ev', 'eye', 'far', 'fe', 'fit', 'fix', 'fiz', 'fm', 'fur', 
           'gap', 'gas', 'gev', 'ghz', 'gr', 'guo', 'han', 'hep', 'hit', 'ho', 'hop', 'hot', 'hu', 
           'hz', 'iff', 'int', 'ion', 'ir', 'jet', 'kev', 'key', 'kim', 'kpc', 'law', 'let', 
           'lhc', 'li', 'lie', 'liu', 'log', 'lot', 'low', 'lu', 'map', 'mat', 'max', 'mev', 
           'mhz', 'mid', 'min', 'mix', 'mm', 'mod', 'mpc', 'ms', 'nee', 'net', 'new', 'nm', 'ns', 
           'nsf', 'odd', 'old', 'opt', 'owe', 'pay', 'pc', 'phd', 'pre', 'qcd', 'raw', 'ray', 
           'red', 'rms', 'row', 'roy', 'run', 'san', 'sci', 'ser', 'set', 'si', 'sit', 'sky', 
           'sm', 'sub', 'sum', 'sun', 'tev', 'tie', 'try', 'uk', 'usa', 'use', 'uv', 'van', 'way', 
           'web', 'wu', 'xu', 'yan', 'yr', 'yu', 'zhu'}

In [ ]:
sw_remove = set(tokens[tokens['token'].apply(lambda t: len(t) <= 3)]['token'].values)
sw_remove = sw_remove.difference(sw_keep)

In [ ]:
new_dct.filter_tokens(bad_ids=[i for i in new_dct if new_dct[i] in sw_remove])

Numbers:

In [ ]:
RE_NUM = re.compile(r'''^(
    zero|one|two|three|four|five|six|seven|eight|nine|ten|
    eleven|twelve|teen|twenty|thirty|fou?rty|fifty|siXty|seventy|eighty|ninety|
    hundred|thousand|million|billion|trillion|quadrillion|quintillion
    )-?\1*$''', re.X)

In [ ]:
tokens[tokens['token'].apply(lambda t: RE_NUM.search(t) is not None)].head()

In [ ]:
nn_remove = set(
    tokens[tokens['token']
           .apply(lambda t: RE_NUM.search(t) is not None)]['token']
           .values)

In [ ]:
new_dct.filter_tokens(bad_ids=[i for i in new_dct if new_dct[i] in nn_remove])

### Lots of Punctuation

In [ ]:
# New token list
tokens = make_token_df(new_dct)

In [ ]:
RE_PUNCT = re.compile(r'[^0-9A-Za-z]')

tokens['len'] = tokens['token'].apply(lambda t: len(t))
tokens['pct'] = tokens['token'].apply(lambda t: len(RE_PUNCT.findall(t)))

In [ ]:
# 4 seems like a good cutoff
tokens[tokens['pct'] == 4].head()

In [ ]:
pct_remove = set(tokens[tokens['pct'] >= 4]['token'].values)
new_dct.filter_tokens(bad_ids=[i for i in new_dct if new_dct[i] in pct_remove])

### Top 25,000 Tokens

 - Similar to other gensim applications and Blei's Correlated Topic Model of Science

In [ ]:
new_dct.filter_extremes(no_above=1.00, keep_n=25000)
new_dct.compactify()

## New Dictionary and Corpus

In [ ]:
new_dct.compactify()

In [ ]:
old_to_new = {
    dct.token2id[token]: token_id
    for token_id, token in new_dct.items()
}

In [ ]:
vt = gensim.models.VocabTransform(old_to_new)
new_corpus = vt[arxiv]

In [ ]:
new_dct.save('arxiv_full_trimmed25K_spacy.mm.dictionary.cpickle')

In [ ]:
gensim.corpora.MmCorpus.serialize('arxiv_full_trimmed25K_spacy.mm', new_corpus)